In [35]:
import json
import os

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats("svg")

In [36]:
file_name = dict(dev="dev.json", test="test.json", train="train.json")

data_path = "./round1_training_data/"

# read data
data = dict()
for key in file_name:
    with open(data_path + file_name[key], "r") as f:
        data[key] = json.load(f)
data["train"][:3]

[{'ID': '0',
  '产品名': '优医保·高端门诊住院医疗',
  '条款': '【中国境内】 指除香港特别行政区、澳门特别行政区以及台湾地区之外的中华人民共和国领土。',
  '问题': '【中国境内】在优医保·高端门诊住院医疗保险产品中指的是哪些地区？',
  '答案': '在优医保·高端门诊住院医疗保险产品中，【中国境内】指的是除香港特别行政区、澳门特别行政区以及台湾地区之外的中华人民共和国领土。'},
 {'ID': '1',
  '产品名': '同佑e生(臻享版)终身重疾险',
  '条款': '中度再生障碍性贫血 指因骨髓造血功能慢性持续性衰竭导致的贫血、中性粒细胞减少及血小板减少。须满足下列至少一项条件,但未达到本合同所指重大疾病“重型再生障碍性贫血”的给付标准：1.骨髓刺激疗法至少一个月；2.免疫抑制剂治疗至少一个月。',
  '问题': '中度再生障碍性贫血会导致哪些症状？',
  '答案': '中度再生障碍性贫血会导致贫血、中性粒细胞减少及血小板减少。'},
 {'ID': '2',
  '产品名': '众民保·百万医疗（免健告）',
  '条款': '认可的互联网药店 经保险人审核认可，取得互联网药品信息服务资格证书，为被保险人提供购药或者配送服务的药店，具体名单在保险人官方网站或销售平台公示。',
  '问题': '互联网药店需要取得哪种证书才能被保险公司认可？',
  '答案': '互联网药店需要取得互联网药品信息服务资格证书才能被保险公司认可。'}]

In [37]:
from langchain_community.chat_models import ChatZhipuAI
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

In [38]:
llm = ChatZhipuAI(
    temperature=0,
    zhipuai_api_key=os.getenv("ZHIPUAI_API_KEY"),
    model_name="GLM-4-0520",
    # model_name="GLM-4-Flash",
)

prompt_template = """使用以下上下文内容来回答后面的问题。如果你不知道答案，就回答你不知道，不要试图编造答案，也不要加入多余的信息。

{context}

问题：{question}
用中文回答：
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain = load_qa_chain(llm, chain_type="stuff", prompt=PROMPT)

In [39]:
context_template = """
产品名：{product_name}
条款：{clause}
"""


def get_query_context(split="train", index=0):

    question = data[split][index]["问题"]
    context = context_template.format(
        product_name=data[split][index]["产品名"], clause=data[split][index]["条款"]
    )
    return dict(question=question, context=context)

In [40]:
llm.invoke(PROMPT.format(**get_query_context("test", 2))).content

'保险条款在太享E保百万医疗险合同中的作用是约定保险权利义务关系，具体明确了保险合同双方的权利和义务，是构成保险合同的重要组成部分。'

In [41]:
PROMPT.format(**get_query_context("test", 2))

'使用以下上下文内容来回答后面的问题。如果你不知道答案，就回答你不知道，不要试图编造答案，也不要加入多余的信息。\n\n\n产品名：太享E保百万医疗险\n条款：合同构成 本合同是您与我们约定保险权利义务关系的协议，包括本保险条款、保险单及其他保险凭证、投保单及其他投保文件、合法有效的声明、批注、批单及其他您与我们共同认可的书面协议。\n\n\n问题：在太享E保百万医疗险合同中，保险条款的作用是什么？\n用中文回答：\n'